# TASK 2
Previsione delle stelle delle recensioni sulla base del testo. 

In [ ]:
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import Embedding
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from official.nlp import optimization

from libraries.dataset import Dataset

import tensorflow.keras as keras
import tensorflow as tf
import tensorflow_text
import keras_tuner as kt

import libraries.preprocessing_utils as prep_utils
import libraries.models_builders as models_builders
import libraries.filenames_generator as filenames  
import constants as const
import pandas as pd

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

print(tf.test.gpu_device_name())

## Data retrieving
Ottenimento dei dati relativi alle recensioni, bilanciati sulla base delle stelle, al fine di ottenere lo stesso numero di review per ogni possibile valutazione (da 1 a 5).
In questo specifico caso, sono richiesti 20'000 campioni per ogni tipo di classe (per un totale di 100'000 campioni).

L'oggetto `review_data` contiene tre field relativi ai subdataset da utilizzare nel progetto:
- `train_data` = tupla contentente i dati ed i target per il training
- `val_data` = tupla contentente i dati ed i target per la validazione
- `test_data` = tupla contentente i dati ed i target per il testing

Alla prima esecuzione, i tre diversi subset sono memorizzati sottoforma di file csv, in modo da evitare la riesecuzione del codice di splitting dei dataset durante le successive esecuzioni.   



In [ ]:
review_data = Dataset('review', 'stars')

review_data.split(['text'], 'stars', n_samples=20_000)

## Data preprocessing comune

Fase di preparazione dei dati grezzi caricati precedentemente, al fine di ottenere testo pulito utilizzabile come base per i diversi modelli di machine learning da testare.

Le azioni di preprocessing attuate in questa fase sono:
  - riduzione testo dal maiuscolo al *minuscolo*
  - *decontrazione* forme contratte
  - rimozione delle stop-words
  - lemmatizzazione

Ogni modello aggiungerà successivamente ulteriori azioni di processamento dei dati, necessarie per adattarli al meglio al tipo di input atteso.


In [ ]:
prep_train_data = prep_utils.preprocess_texts(review_data.train_data[0]['text'], path= filenames.picked_cleaned_sentences(
        "train", "task2"))

prep_test_data = prep_utils.preprocess_texts(review_data.test_data[0]['text'], path= filenames.picked_cleaned_sentences(
        "test", "task2"))

prep_val_data = prep_utils.preprocess_texts(review_data.val_data[0]['text'], path= filenames.picked_cleaned_sentences(
        "val", "task2"))

## Training dei classificatori
Fase di addestramento dei classificatori.

I classificatori testati sono:
- **Multinomial Naive Bayes**
- **KNN**
- **SVM**
- **LSTM**
- **BERT**

### Multinomial Naive Bayes

Addestramento del classificatore di tipo Multinomial Naive Baye su un subset ridotto di 30'000 campioni.

E' stata applicata ai dati un'azione aggiuntiva di preprocessing, eseguita dall'oggetto `CountVectorizer`, il quale converte la collezione dei testi delle recensioni in una matrice contenente il conteggio dei token.

In [4]:
sub_train_df = pd.DataFrame({'text': prep_train_data, 'stars': review_data.train_data[1]})
s1 = sub_train_df.loc[sub_train_df['stars'] == 1].iloc[:6000, :]
s2 = sub_train_df.loc[sub_train_df['stars'] == 2].iloc[:6000, :]
s3 = sub_train_df.loc[sub_train_df['stars'] == 3].iloc[:6000, :]
s4 = sub_train_df.loc[sub_train_df['stars'] == 4].iloc[:6000, :]
s5 = sub_train_df.loc[sub_train_df['stars'] == 5].iloc[:6000, :]

sub_train_df = pd.concat([s1, s2, s3, s4, s5], ignore_index=True)
sub_train_df = sub_train_df.sample(frac=1, random_state=const.seed)

In [5]:
vectorizer = CountVectorizer()
nb_train_data = vectorizer.fit_transform(sub_train_df['text']).toarray()

In [ ]:
nb_model = MultinomialNB()
nb_model.fit(nb_train_data, sub_train_df['stars'])

Nella fase di testing del modello addestrato, l'accuratezza raggiunta è stata del 51.6% sul test set.

In [7]:
nb_test_data = vectorizer.transform(prep_test_data[:10000]).toarray()
nb_model.score(nb_test_data, review_data.test_data[1][:10000])

0.516

### KNN

Addestramento del classificatore di tipo KNN su un subset ridotto di 30'000 campioni.

Sono stati utilizzati gli stessi input processati per il modello Naive Bayes precedente ed il miglior modello è stato ricercato eseguendo diverse prove con valori per il numero di vicini.

L'accuratezza raggiunta nella fase di testing del modello è stata del 35.3% ed il valore di *k* che ha permesso di raggiungere tale percentuale è stato di 50.

In [ ]:
cv = GridSearchCV(KNeighborsClassifier(), param_grid={"n_neighbors": [2,4,10,15,25,50,100,200,400]})
cv.fit(nb_train_data, sub_train_df['stars'])

In [9]:
nb_test_data = vectorizer.transform(prep_test_data[:10000]).toarray()
print(cv.score(nb_test_data, review_data.test_data[1][:10000]))
print(cv.best_params_)

0.353
{'n_neighbors': 50}


### SVM
Addestramento del classificatore di tipo SVM su un subset ridotto di 2'000 campioni.

E' stata applicata ai dati un'azione aggiuntiva di preprocessing, eseguita dall'oggetto `CountVectorizer` e sono state testate diverse architetture con valori del parametro C differenti.

L'accuratezza raggiunta nella fase di testing del modello è stata del 43.4% ed il valore di C che ha permesso di raggiungere tale percentuale è pari a 1 con l'utilizzo di un kernel lineare.

In [10]:
sub_train_df = pd.DataFrame({'text': prep_train_data, 'stars': review_data.train_data[1]})
s1 = sub_train_df.loc[sub_train_df['stars'] == 1].iloc[:400, :]
s2 = sub_train_df.loc[sub_train_df['stars'] == 2].iloc[:400, :]
s3 = sub_train_df.loc[sub_train_df['stars'] == 3].iloc[:400, :]
s4 = sub_train_df.loc[sub_train_df['stars'] == 4].iloc[:400, :]
s5 = sub_train_df.loc[sub_train_df['stars'] == 5].iloc[:400, :]

sub_train_df = pd.concat([s1, s2, s3, s4, s5], ignore_index=True)
sub_train_df = sub_train_df.sample(frac=1, random_state=const.seed)

In [11]:
svm_train_data = vectorizer.fit_transform(sub_train_df['text']).toarray()
cv = GridSearchCV(svm.SVC(), param_grid={"kernel": ['linear'],"C": [1,2,4,10,15,25,50,100,200,400]})
cv.fit(svm_train_data, sub_train_df['stars'])

In [ ]:
svm_test_data = vectorizer.transform(prep_test_data[:750]).toarray()
print(cv.score(svm_test_data, review_data.test_data[1][:750]))
print(cv.best_params_)

## LSTM
Al fine di trovare la combinazione di iperparametri che rendono le migliori performance, sono state testate diverse architetture delle **Reti neurali ricorrenti** di tipo **LSTM**.

Anche in questo caso è stato effettuato un preprocessing aggiuntivo sui testi delle recensioni (i quali fungono come input alle reti).
Tale attività è stata eseguita da un *tokenizer* specifico, il quale restituisce, per ogni recensione, un vettore contenente gli indici delle parole in essa contenute.

Tali indici sono relativi alla posizione delle parole nel dizionario estratto precedemente dalla collezione di recensioni e sono fondamentali per il recupero delle word embedding corrispondenti alle parole, come avviene nel layer di tipo **Embedding** impostato come primo layer della rete LSTM.

In [4]:
tokenizer = prep_utils.get_tokenizer(review_data.train_data[0]['text'])

train_tokens = prep_utils.get_set_tokens(
    review_data.train_data[0]['text'], tokenizer, set='train', task='task2')

test_tokens = prep_utils.get_set_tokens(
    review_data.test_data[0]['text'], tokenizer, set='test', task='task2')

val_tokens = prep_utils.get_set_tokens(
    review_data.val_data[0]['text'], tokenizer, set='val', task='task2')

Esistono diverse metodologie per la definizione dei **word vectors** e queste spaziano dal training di reti come *Word2Vec* all'utilizzo di mapping già pre-addestrati.

Nel caso di questo studio è stato scelto di creare un'embedding matrix a partire da un mapping già esistente, nello specifico quello messo a disposizione da **Glove** e addestrato su una grande mole di dati testuali estratti da *twitter*.

Sulla base di questo mapping, sono stati estratti, ed inseriti in una matrice, i vettori delle parole presenti nel dizionario e, se non presenti, queste sono state rappresentate come 0-vector.

In [ ]:
e_matrix = prep_utils.get_embedding_matrix(const.word_embedding_filepath, 'task2',
                                            tokenizer, len(tokenizer.index_word)+1)

word_vector_dim = 100

vocab_size = len(tokenizer.word_index) +1
max_length = len(max(train_tokens, key=len))

embedding_layer = Embedding(vocab_size, word_vector_dim,
                            embeddings_initializer=Constant(e_matrix), trainable=False)

Fase di tuning degli iperparametri.

La ricerca è stata condotta su un unico trial. Gli iperparametri testati sono:
- *numero di unità* (dimensione del vettore delle celle e hidden states)
- *percentuale di dropout*
- *learning rate*

Il training è stato gestito utilizzando la tecnica dell'*early stopping*, dettata dalla seguente callback, la quale termina il training dopo 15 epoche prive di miglioramenti.

In [6]:
stop_early_cb = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=15)

In [ ]:
project_name = "task2_lstm_adam_128"

builder = models_builders.get_rnn_builder(
    drop=[0.2, 0.5],
    units=[15, 20, 50, 80],
    lrate=[0.01, 0.001],
    optimizer=keras.optimizers.Adam,
    embedding_layer=embedding_layer,
    output_shape=5,
    activation="softmax",
    loss=keras.losses.CategoricalCrossentropy())

tuner = kt.RandomSearch(
    builder,
    objective = 'val_accuracy',
    max_trials = 10,
    directory = const.tuner_path, project_name = project_name
)

tuner.search_space_summary()

In [ ]:
tuner.search(train_tokens, pd.get_dummies(review_data.train_data[1]),
             batch_size=128, epochs=1000,
             validation_data=(val_tokens, pd.get_dummies(review_data.val_data[1])),
             callbacks=[
                 stop_early_cb,
                 tf.keras.callbacks.TensorBoard(const.logs_path + project_name, update_freq='epoch')],
             verbose=0)

Il miglior modello trovato, in seguito all'esecuzione dell'unico trial, fornisce un'accuratezza sul validation set del 60.5% e si presenta come una rete con le seguenti caratteristiche:
- dropout del 20%
- learning rate di 0.01
- 15 units

Tutte le statistiche di esecuzione sono visualizzabili su tensorboard.

## BERT
E' stata testata anche la tecnica **BERT**, basata sui transformer.

Si è scelto di basarsi sul modello pre-addestrato **Small Bert**, caratterizzato da un numero minore di transformer blocks, e di procedere successivamente con il suo fine-tuning per adattarlo al problema.

In [5]:
handle_encoder = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'
handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

Ricaricamento del dataset e suddivisione in training, validation e test set con percentuali differenti rispetto a prima. In questo caso è stato assegnato l'80% dei campioni al training set ed il restante 20% al validation e testing set.

In [ ]:
review_data = Dataset('review', 'stars')
review_data.split(['text'], 'stars', n_samples=20_000, val_size=0.1, test_size=0.1)

In [ ]:
prep_train_data = prep_utils.preprocess_texts(review_data.train_data[0]['text'], path= filenames.picked_cleaned_sentences(
        "train", "task2"))

prep_test_data = prep_utils.preprocess_texts(review_data.test_data[0]['text'], path= filenames.picked_cleaned_sentences(
        "test", "task2"))

prep_val_data = prep_utils.preprocess_texts(review_data.val_data[0]['text'], path= filenames.picked_cleaned_sentences(
        "val", "task2"))

Processamento aggiuntivo dei dati e wrapping dei testi delle recensioni e dei target nell'oggetto `Dataset`.

In [8]:
train_df = tf.data.Dataset.from_tensor_slices((prep_train_data, pd.get_dummies(review_data.train_data[1])))
val_df = tf.data.Dataset.from_tensor_slices((prep_val_data, pd.get_dummies(review_data.val_data[1])))

Creazione del modello.

In [9]:
epochs = 5

steps_per_epoch = tf.data.experimental.cardinality(train_df).numpy()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps, # lr decay
                                          optimizer_type='adamw')

In [10]:
model = models_builders.build_BERT_model(handle_preprocess, handle_encoder, 5, activation="softmax")

model.compile(optimizer=optimizer,
                loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics=tf.metrics.CategoricalAccuracy())

Addestramento del modello.

L'accuratezza raggiunta sul training set è del 62.6% e sul validation set del 59.3%. 

In [ ]:
batch = 16

history = model.fit(x=train_df.batch(batch),
                    validation_data=val_df.batch(batch),
                    epochs=epochs,
                    callbacks=[
                               tf.keras.callbacks.TensorBoard(const.logs_path + "bert_task2", update_freq='epoch')])

Il modello sul test set ha raggiunto un'accuratezza del 58.8%.

In [12]:
test_df = tf.data.Dataset.from_tensor_slices((prep_test_data, pd.get_dummies(review_data.test_data[1])))

loss, accuracy = model.evaluate(test_df.batch(1))

1000/1000 [==============================] - 95s 94ms/step - loss: 1.0897 - categorical_accuracy: 0.5880
